A notebook that allows manual interactions with an RTSP server.  
Intended to increase one's understanding of the RTSP protocol.

In [1]:
import socket
import hashlib
from urllib.parse import urlparse

In [2]:
# Function to read RTSP URLs from a file
def read_rtsp_urls(filename):
    urls = []
    try:
        with open(filename, 'r') as file:
            urls = [line.strip() for line in file.readlines() if line.strip()]
    except FileNotFoundError:
        print(f"Error: File {filename} not found.")
    return urls

In [3]:
def send_rtsp_options(url):
    # Parse the RTSP URL to extract IP address and port
    parsed_url = urlparse(url)
    server_ip = parsed_url.hostname
    server_port = parsed_url.port if parsed_url.port else 554
    
    # Create a TCP socket
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        # Connect to the RTSP server
        client_socket.connect((server_ip, server_port))
        
        # Prepare the RTSP OPTIONS request
        cseq = 1
        options_request = f"OPTIONS {url} RTSP/1.0\r\n"
        options_request += f"CSeq: {cseq}\r\n"
        options_request += "User-Agent: Python RTSP Client\r\n"
        options_request += "\r\n"
        
        # Send the request to the RTSP server
        client_socket.send(options_request.encode())
        
        # Receive the response from the server
        response = client_socket.recv(4096)
        print("RTSP Server Response:\n")
        print(response.decode())
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Close the socket connection
        client_socket.close()

In [4]:
def send_rtsp_describe(url):
    # Parse the RTSP URL to extract IP address, port, username, and password
    parsed_url = urlparse(url)
    server_ip = parsed_url.hostname
    server_port = parsed_url.port if parsed_url.port else 554
    username = parsed_url.username
    password = parsed_url.password
    
    # Create a TCP socket
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        # Connect to the RTSP server
        client_socket.connect((server_ip, server_port))
        
        # Prepare the initial RTSP DESCRIBE request
        cseq = 2
        describe_request = f"DESCRIBE {url} RTSP/1.0\r\n"
        describe_request += f"CSeq: {cseq}\r\n"
        describe_request += "User-Agent: Python RTSP Client\r\n"
        describe_request += "Accept: application/sdp\r\n"
        describe_request += "\r\n"
        
        # Send the request to the RTSP server
        client_socket.send(describe_request.encode())
        
        # Receive the response from the server
        response = client_socket.recv(4096)
        response_str = response.decode()
        print("RTSP Server Response (DESCRIBE):\n")
        print(response_str)
        
        # Check if authentication is required (401 Unauthorized)
        if "401 Unauthorized" in response_str:
            # Extract the realm and nonce from the response
            realm = None
            nonce = None
            for line in response_str.split("\r\n"):
                if line.startswith("WWW-Authenticate: Digest"):
                    parts = line.split(",")
                    for part in parts:
                        if "realm" in part:
                            realm = part.split('"')[1]
                        elif "nonce" in part:
                            nonce = part.split('"')[1]
            
            if realm and nonce and username and password:
                # Compute the digest response
                ha1 = hashlib.md5(f"{username}:{realm}:{password}".encode()).hexdigest()
                ha2 = hashlib.md5(f"DESCRIBE:{url}".encode()).hexdigest()
                response_digest = hashlib.md5(f"{ha1}:{nonce}:{ha2}".encode()).hexdigest()
                
                # Prepare the authenticated DESCRIBE request
                cseq += 1
                describe_request = f"DESCRIBE {url} RTSP/1.0\r\n"
                describe_request += f"CSeq: {cseq}\r\n"
                describe_request += "User-Agent: Python RTSP Client\r\n"
                describe_request += "Accept: application/sdp\r\n"
                describe_request += (
                    f"Authorization: Digest username=\"{username}\", realm=\"{realm}\", nonce=\"{nonce}\", uri=\"{url}\", response=\"{response_digest}\"\r\n"
                )
                describe_request += "\r\n"
                
                # Send the authenticated request to the RTSP server
                client_socket.send(describe_request.encode())
                
                # Receive the response from the server
                response = client_socket.recv(4096)
                print("RTSP Server Response (Authenticated DESCRIBE):\n")
                print(response.decode())
            else:
                print("Authentication required, but realm/nonce extraction failed or credentials not provided.")
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Close the socket connection
        client_socket.close()



In [5]:
rtsp_urls = read_rtsp_urls('rtsp_urls.txt')

In [ ]:
#send_rtsp_options(rtsp_urls[1][1:-1])
send_rtsp_describe(rtsp_urls[1][1:-1])

In [ ]:
#send_rtsp_options("rtsp://192.168.50.67:8080/video/h264")
send_rtsp_describe("rtsp://192.168.50.67:8080/video/h264")